In [170]:
!pip install opencv-python-headless
!pip install opencv-python
!pip install scipy
!pip3 install pandas

In [2]:
import scipy.io
import cv2
import pandas as pd
import os
import random 
import numpy as np
import json
import math

In [118]:
def parseVideoName(video_name):
    video_name = video_name.replace('.avi', '')
    first_part,second_part = video_name.split(',')
    video_type, num_targets = first_part.rsplit('_', 1)
    videos_info = second_part.split('_')
    
    parsed_data = {
        'video_name': video_name,
        'target_name': second_part,
        'video_type': video_type,
        'num_targets': int(num_targets),
        'first_video': int(videos_info[0]),
        'second_video': int(videos_info[1]),
        'first_start_frame': int(videos_info[2]),
        'second_start_frame': int(videos_info[3]),
        'first_rotation': float(videos_info[4]),
        'second_rotation': float(videos_info[5])
    }
    
    return parsed_data
    
def generateRandomColor():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def processTrialsVideos(file_path):
    df = pd.read_csv(file_path)
    parsed_rows = []
    
    for index, row in df.iterrows():
        video_name = row['videoName']
        parsed_data = parseVideoName(video_name)
        parsed_data['firstTargets'] = row['firstTargets']
        parsed_data['secondTargets'] = row['secondTargets']
        parsed_data['fullRotation'] = row['rotAngle']
        parsed_rows.append(parsed_data)
        # parsed_rows.append(1)
    return pd.DataFrame(parsed_rows)
    
def loadMatFile():
    root_folder = 'GazeMatV21'
    test_subs = ['sub11','sub21','sub31','sub41','sub51']

    data = {}
    
    for sub_dir in os.listdir(root_folder):
        if sub_dir in test_subs:
            mat_file_path = os.path.join(root_folder, sub_dir, 'finalSampled.mat') # 40
            if os.path.exists(mat_file_path):
                mat_data = scipy.io.loadmat(mat_file_path)
                data[sub_dir] = mat_data['itemsSampled']
            else:
                print(f"{mat_file_path} does not exist.")
    print("Data loaded successfully.")
    return data


def loadShapeFishPoints(videoName,numberFrame):
    json_filename = f'img{numberFrame:03d}.json'
    json_path = os.path.join('./jsons', str(videoName), json_filename)

    if not os.path.exists(json_path):
        raise FileNotFoundError(f"JSON file not found: {json_path}")

    with open(json_path, 'r') as json_file:
        data = json.load(json_file)
    
    return data.get('shapes', [])

def getTargetsFromJson(shapes,targets):
    if not isinstance(targets, (list, tuple)):
        targets = [targets]
    
    target_digits = set()
    for target in targets:
        if isinstance(target, int):
            target_digits.update(str(target))
        else:
            raise ValueError("Targets should be integers or lists/tuples of integers")

    filtered_shapes = [shape for shape in shapes if shape['label'] in target_digits]
    
    return filtered_shapes

def getTotalShape(shapeInfo):
    firstFrame = shapeInfo['firstStart']
    secondFrame = shapeInfo['secondStart']
    firstTargets = shapeInfo['firstTargets']
    secondTargets = shapeInfo['secondTargets']
    shapes_video1 = loadShapeFishPoints(shapeInfo['first'], firstFrame)
    shapes_video2 = loadShapeFishPoints(shapeInfo['second'], secondFrame)
    merged_shapes =  getTargetsFromJson(shapes_video1,firstTargets) + getTargetsFromJson(shapes_video2,secondTargets)
    # print(merged_shapes)
    return merged_shapes

def generateSubjectName(subIndex):
    number_part = subIndex[3:]
    modified_number = number_part[:-1] if len(number_part) > 1 else '0'
    return f'sub{modified_number}'

def getVideoInfo(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # num_frames = int(mainVideo.get(cv2.CAP_PROP_FRAME_COUNT))
    return fps, width, height 

def createOutputVideo(video,name):
    output_gaze_videos = 'gazeOutputVideo'
    os.makedirs(output_gaze_videos, exist_ok=True)
    fps, width, height = getVideoInfo(video)
    
    output_gaze_videos_path = os.path.join(output_gaze_videos, name + '.avi')
    output_video = cv2.VideoWriter(output_gaze_videos_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
    return output_video

def getSubjectColor(subject):
    if subject not in subject_colors:
        subject_colors[subject] = generateRandomColor()
    return subject_colors[subIndex]

def mapPointsOnFrame(frame):
    return round(frame * (7798 / 110))

def openVideo(video):
    trialVideo = cv2.VideoCapture(os.path.join('./videos', video.video_type, video.target_name + '.avi'))
    
    if not trialVideo.isOpened():
        raise ValueError(f"Unable to open video file: {os.path.join('./videos', video.video_type, video.target_name + '.avi')}")
    return trialVideo

def generateXPoints(x):
    # scale_x = 660 / 1920
    # return int(x * scale_x)
    return int(x) - 630

def generateYPoints(y):
    # scale_y = 660 / 1080
    # return int(y * scale_y)
    return int(y) - 210


def getTotalShapeFirst(shapeInfo,baseFrame):
    firstFrame = shapeInfo['firstStart'] + baseFrame
    firstTargets = shapeInfo['firstTargets']
    shapes_video1 = loadShapeFishPoints(shapeInfo['first'], firstFrame)
    return getTargetsFromJson(shapes_video1,firstTargets)
    
def getTotalShapeSecond(shapeInfo,baseFrame):
    secondFrame = shapeInfo['secondStart'] + baseFrame
    secondTargets = shapeInfo['secondTargets']
    shapes_video2 = loadShapeFishPoints(shapeInfo['second'], secondFrame)
    merged_shapes = getTargetsFromJson(shapes_video2,secondTargets)
    return merged_shapes

def rotate_frame(frame, angle, center):
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_frame = cv2.warpAffine(frame, M, (frame.shape[1], frame.shape[0]))
    return rotated_frame

def adjust_shape_points(shape, x_offset=90, y_offset=10):
    adjusted_points = []
    for point in shape['points']:
        adjusted_point = [point[0] - x_offset, point[1] - y_offset]
        adjusted_points.append(adjusted_point)
    shape['points'] = adjusted_points
    return shape
    
def draw_shapes(frame, shapes, typeV, color=(0, 0, 255)):
    data = {
        1: [90, 5],
        7: [25,25],
        10: [25,15]
    }
    overlay = frame.copy()
    opacity = 0.1
    for shape in shapes:
        shape = adjust_shape_points(shape,data[typeV][0],data[typeV][1])  # Adjust the points
        points = shape['points']

        if not points:
            continue
        
        np_points = np.array(points, dtype=np.int32).reshape((-1, 1, 2))

        if np_points.size == 0:
            continue
        # x, y = np_points[0][0]  # Position for the text
        # cv2.putText(frame, f"_{typeV}", (x + 20, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 225), 1, cv2.LINE_AA)
        cv2.polylines(overlay, [np_points], isClosed=True, color=color, thickness=1)
        cv2.fillPoly(overlay, [np_points], color=color)
    cv2.addWeighted(overlay, opacity, frame, 1 - opacity, 0, frame)

def rotate_frameNew(frame, angle):
    center = (330, 330)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_frame = cv2.warpAffine(frame, M, (frame.shape[1], frame.shape[0]))
    return rotated_frame

def loadAndProcessData():
    root_folder = 'GazeMatV21'
    test_subs = ['sub11', 'sub21', 'sub31', 'sub41', 'sub51']

    data = {}

    for sub_dir in os.listdir(root_folder):
        if sub_dir in test_subs:
            csv_file_path = f'./{sub_dir}.csv' 
            
            video_data = pd.DataFrame()
            
            if os.path.exists(csv_file_path):
                video_data = processTrialsVideos(csv_file_path)
            else:
                print(f"{csv_file_path} does not exist.")
            
            data[sub_dir] = video_data
    
    print("Data loaded and processed successfully.")
    return data

In [119]:
subjects = loadMatFile()
subject_colors = {}
subjects['sub21']

Data loaded successfully.


array([[array([[1240.5       ],
               [1201.09997559],
               [1199.30004883],
               [1141.40002441],
               [ 950.5       ],
               [ 954.5       ],
               [1046.59997559],
               [1061.        ],
               [1164.19995117],
               [1144.90002441],
               [1143.90002441],
               [1141.30004883],
               [1104.5       ],
               [1100.40002441],
               [1096.30004883],
               [1094.59997559],
               [1144.80004883],
               [1216.        ],
               [1220.59997559],
               [1218.30004883],
               [1219.40002441],
               [1220.09997559],
               [1195.90002441],
               [1200.30004883],
               [1138.09997559],
               [1147.30004883],
               [1190.90002441],
               [1198.30004883],
               [1200.30004883],
               [1205.59997559],
               [1204.69995117],
        

In [120]:
### to Check data 

for index in subjects:
    x_points = subjects[index][0]  # First column for x points
    y_points = subjects[index][1]
# print(check_all_elements_type(subjects['sub11'][1][0]))
trialNumber  = 0

# Pay attentoion: it is from 0 to 39
xc = subjects['sub21'][trialNumber][0] # 110
yc = subjects['sub21'][trialNumber][1]

print(xc[0][0])

1240.5


In [123]:
csvSample = processTrialsVideos('./sub21.csv') # 21
data = loadAndProcessData()
# We are checking the sub11 gaze mat file and its trials
# so we should see sub11 csv file
# detailsCSV['sub21']
data['sub11']

Data loaded and processed successfully.


,video_name,target_name,video_type,num_targets,first_video,second_video,first_start_frame,second_start_frame,first_rotation,second_rotation,firstTargets,secondTargets,fullRotation
0,"NO_2,1_10_1_1_45_315",1_10_1_1_45_315,NO,2,1,10,1,1,45.0,315.0,12,0,278
1,"NO_2,1_10_119_344_45_90",1_10_119_344_45_90,NO,2,1,10,119,344,45.0,90.0,3,4,117
2,"OB_4,1_10_1_344_45_315",1_10_1_344_45_315,OB,4,1,10,1,344,45.0,315.0,3412,0,283
3,"OB_2,10_10_114_229_0_135",10_10_114_229_0_135,OB,2,10,10,114,229,0.0,135.0,14,0,170
4,"NO_4,10_10_229_344_0_315",10_10_229_344_0_315,NO,4,10,10,229,344,0.0,315.0,43,41,13
5,"OB_2,1_10_1_1_45_0",1_10_1_1_45_0,OB,2,1,10,1,1,45.0,0.0,0,14,64
6,"OC_4,1_10_119_344_0_247.5",1_10_119_344_0_247.5,OC,4,1,10,119,344,0.0,247.5,13,23,260
7,"OB_OC_4,1_10_119_114_45_225",1_10_119_114_45_225,OB_OC,4,1,10,119,114,45.0,225.0,342,1,171
8,"OC_4,1_10_119_344_45_90",1_10_119_344_45_90,OC,4,1,10,119,344,45.0,90.0,31,41,55
9,"OB_4,1_10_119_229_45_225",1_10_119_229_45_225,OB,4,1,10,119,229,45.0,225.0,342,4,123


In [116]:
videoList = []
trialNumber  = 0

for sub_dir, csvSample in data.items():
    previous_x = None
    previous_y = None
    for index, video in csvSample.iterrows():
        try:
            
            open_video = openVideo(video)
            output_video_name = f"{sub_dir}_{video.video_name}"
            output_video = createOutputVideo(open_video,output_video_name)
            
            frame_idx = 0
    
            while True:
                ret, frame = open_video.read()
                if not ret:
                    break  
    
                frame = rotate_frameNew(frame, video['fullRotation'])
                for subIndex in subjects:
                    x_points = subjects[subIndex][trialNumber][0]  # (7798, 1)
                    y_points = subjects[subIndex][trialNumber][1]  # (7798, 1)
    
                    x = x_points[frame_idx][0] # the index 0 is because of the structure, they have just one item 
                    y = y_points[frame_idx][0] 
    
                    if math.isnan(x):
                        if previous_x is not None:
                            x = previous_x
                        else:
                            print(f"NaN detected for x at frame {frame_idx}, but no previous value to replace. Skipping.")
                            continue
    
                    if math.isnan(y):
                        if previous_y is not None:
                            y = previous_y
                        else:
                            print(f"NaN detected for y at frame {frame_idx}, but no previous value to replace. Skipping.")
                            continue
            
                    previous_x = x
                    previous_y = y
                    
                    new_x = generateXPoints(x)
                    new_y = generateYPoints(y)
                    
                    color = getSubjectColor(subIndex)
    
                    cv2.circle(frame, (new_x, new_y), 5, color, 3) 
    
                    cv2.putText(frame, generateSubjectName(subIndex), (new_x + 5, new_y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
    
                shapeInfo = {
                    'first': video.first_video,
                    'second': video.second_video,
                    'firstTargets': video.firstTargets,
                    'secondTargets': video.secondTargets,
                    'firstStart': video.first_start_frame,
                    'secondStart': video.second_start_frame
                }
    
                shapeFirst = getTotalShapeFirst(shapeInfo, frame_idx)
                shapeSecond = getTotalShapeSecond(shapeInfo, frame_idx)
                
                blank_frame = np.zeros_like(frame)
                blank_frame2 = np.zeros_like(frame)
                draw_shapes(blank_frame, shapeFirst,video.first_video)
                draw_shapes(blank_frame2, shapeSecond,video.second_video)
    
                center = (330,330)
                rotated_shapes_frame = rotate_frame(blank_frame, video.first_rotation, center)
                full_rotated_first = rotate_frameNew(rotated_shapes_frame, video['fullRotation'])
                rotated_shapes_frame2 = rotate_frame(blank_frame2, video.second_rotation, center)
                full_rotated_second = rotate_frameNew(rotated_shapes_frame2, video['fullRotation'])
    
                mask = full_rotated_first > 0
                frame[mask] = full_rotated_first[mask]
    
                mask2 = full_rotated_second > 0
                frame[mask2] = full_rotated_second[mask2]
                
                
                resized_frame = cv2.resize(frame, (660, 660))
                output_video.write(resized_frame)
                
                frame_idx += 1
            
            trialNumber += 1
            
        except Exception as e:
            print(f"An error occurred while processing video at index {index}: {e}")
    
        finally:
            # Release everything
            if 'open_video' in locals() and open_video.isOpened():
                open_video.release()
            if 'output_video' in locals():
                output_video.release()
            cv2.destroyAllWindows()
    

An error occurred while processing video at index 0: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 1: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 2: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 3: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 4: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 5: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 6: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 7: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 8: index 40 is out of bounds for axis 0 with size 40
An error occurred while processing video at index 9: index 40 is out of b